In [1]:
import os
import shutil

In [2]:
%pwd

'd:\\Artificial Intelligence and Machine Learning\\End2End Projects\\chicken-disease-classification\\research'

In [3]:
os.chdir("../")
%pwd

'd:\\Artificial Intelligence and Machine Learning\\End2End Projects\\chicken-disease-classification'

In [26]:
## Define Entity for Preparing the Model
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_input_shape: list
    params_batch_size : int
    params_num_classes : int
    params_weights: str
    params_include_top: bool
    params_learning_rate: float
    params_horizontal_flip: bool
    params_rotation_range: float
    params_zoom_range: float
    params_epochs: int
    params_dropout_rate: float
    params_weight_decay: float
    params_freeze_all: bool
    params_freeze_till: int


In [27]:
## Update Configuration Manager
from src.cnnclassifier.constants import *
from src.cnnclassifier.utils.common import *

In [28]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 secret_filepath = SECRET_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.secret = read_yaml(secret_filepath)

        

        create_directories_files([self.config.artifacts_root])

    def get_prepare_base_model_config(self)->PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        create_directories_files([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(root_dir = Path(config.root_dir), 
                                                           base_model_path = Path(config.base_model_path),
                                                           updated_base_model_path = Path(config.updated_base_model_path),
                                                           params_input_shape = self.params.input_shape,
                                                           params_batch_size = self.params.batch_size,
                                                           params_num_classes = self.params.num_classes,
                                                           params_weights = self.params.weights,
                                                           params_include_top = self.params.include_top,
                                                           params_learning_rate = self.params.learning_rate,
                                                           params_horizontal_flip = self.params.horizontal_flip,
                                                           params_rotation_range = self.params.rotation_range,
                                                           params_zoom_range = self.params.zoom_range,
                                                           params_epochs = self.params.epochs,
                                                           params_dropout_rate = self.params.dropout_rate,
                                                           params_weight_decay = self.params.weight_decay,
                                                           params_freeze_all = self.params.freeze_all,
                                                           params_freeze_till = self.params.freeze_till
                                                           )
        
        return prepare_base_model_config




In [29]:
## Update Components
import os
import tensorflow as tf

In [30]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(include_top = self.config.params_include_top,
                                                       weights = self.config.params_weights,
                                                       input_shape = self.config.params_input_shape
                                                       )
        
        self.save_model(path = self.config.base_model_path, model = self.model)

    ## Static Method is defined when we want to use a function that doesn't rely on the class
    ## We cannot use self when a function is defined as static method
    ## Static Function can be directly run without instantating the class. Use it directly as
    ## PrepareBaseModel._prepare_full_model()

    @staticmethod
    def _prepare_full_model(model, classes:int, freeze_all:bool, freeze_till:int, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till>0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
        flatten_in = tf.keras.layers.Flatten()(model.output)
        dense1 = tf.keras.layers.Dense(units = 2048, activation = "relu")(flatten_in)
        dense2 = tf.keras.layers.Dense(units = 128, activation = "relu")(dense1)
        prediction = tf.keras.layers.Dense(units = classes,
                                           activation = "softmax")(dense2)
        full_model = tf.keras.models.Model(model.input, 
                                           prediction)
        full_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate),
                           loss = tf.keras.losses.CategoricalCrossentropy(),
                           metrics = ["accuracy", "precision", "recall"])
        
        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(model = self.model, 
                                                   classes = self.config.params_num_classes, 
                                                   freeze_all = self.config.params_freeze_all, 
                                                   freeze_till = self.config.params_freeze_till, 
                                                   learning_rate = self.config.params_learning_rate)
        self.save_model(path = self.config.updated_base_model_path, 
                        model = self.full_model)
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(filepath = path)



    

In [31]:
## Create Pipeline
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config = prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-08-29 08:23:38,692: INFO : common : config\config.yaml loaded successfully]
[2024-08-29 08:23:38,696: INFO : common : params\params.yaml loaded successfully]
[2024-08-29 08:23:38,698: INFO : common : secrets\secrets.yaml loaded successfully]
[2024-08-29 08:23:38,699: INFO : common : Parent Directories and Files Successfully Created]
[2024-08-29 08:23:38,700: INFO : common : Parent Directories and Files Successfully Created]
[2024-08-29 08:23:39,099: WARNING : saving_utils : Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                